# Medidas de actividad por electrodo: Laplaciano de la señal.

Veamos. Franco dice que hay una distribucion trimodal en los datos. Cada moda indica la superposición de otra señal. Una es el ruido de fondo. La mas violenta debe ser la actividad real.

In [1]:
using MAT 
DatosAparato=matopen("propagacion.mat");
matread("propagacion.mat")

Dict{ASCIIString,Any} with 4 entries:
  "SamplingFrequency" => 7022.0
  "StopFrame"         => 5.549486e6
  "StartFrame"        => 5.545273e6
  "RawMatrix"         => [4125.0 2.01416015625 -28.1982421875 -10.07080078125 -…

La siguiente variable contiene TODO el arreglo de los datos, cuyas longitudes son espacio, espacio y tiempo.

In [2]:
LasMedicionesCrudas=read(DatosAparato, "RawMatrix");
size(LasMedicionesCrudas)

(64,64,4213)

Como podemos ver, los datos fueron tomados a 7022 muestras por segundo, son 4213 cuadros, y cada cuadro es de 64 por 64 entradas. Estas estan medidas en microvolts, $\mu V$, respecto a una tierra base que tiene el aparato ahí definida.

La actividad mas interesante se da entre los cuadros 1300 y 2700 aprox, vamos a usar solo esos.

In [3]:
ParteInteresante=LasMedicionesCrudas[:,:,1300:2700];
medidasdatos=size(ParteInteresante)
tmax=medidasdatos[3]

1401

Hay un par de electros saturados, que marcan voltajes absurdamente altos. Esos los vamos a matar con una funcion umbral. La funcion umbral no va a matar señales pequeñas. Eso es tarea de una suavización temporal. Despues de todo, el ruido es temporal.

In [4]:
function umbral(x)
    result=((abs(x)>150) ? 0 : x)
end;

function aplastar(x)
    result=((abs(x)<40) ? 0 : x)
end;

function UnNormGauss(x,sigma)
    return exp(-x*x/(2*sigma))
end


UnNormGauss (generic function with 1 method)

In [5]:
map!(umbral, ParteInteresante); #map! quiere decir map in situ
DesviacionPorElectrodo=zeros(64,64)
for j=1:64, k=1:64
    DesviacionPorElectrodo[j,k]=std(reshape(ParteInteresante[j,k,:],tmax));
end

In [6]:
include("Otsu01.jl")


OtsuUmbralizar (generic function with 1 method)

In [7]:
MascaraOtsu=OtsuUmbralizar(DesviacionPorElectrodo);

In [8]:
canalesinteresantes=sum(MascaraOtsu)

1128

In [9]:
CuernoDeAmon=zeros(ParteInteresante);
for t=1:tmax
    CuernoDeAmon[:,:,t]=ParteInteresante[:,:,t].*MascaraOtsu
end

In [10]:
function PromedioSuavizar(Datos, mediaventana)
    #Bien, veamos como chingaos le ponemos "padding" a las convoluciones.
    colchon=zeros(mediaventana)
    result=zeros(Datos)
    datoscolchon=vcat(colchon, Datos, colchon)
    for t=mediaventana+1:length(Datos)-mediaventana
        result[t-mediaventana]=mean(datoscolchon[t-mediaventana:t+mediaventana])
    end
    return result
end
     

PromedioSuavizar (generic function with 1 method)

In [11]:
function GaussSuavizar(Datos,Sigma)  
    #sigma esta medido en pixeles, es la desviacion estandar de nuestro kernel.
    #El medioancho de nuestra ventana seran 3 sigmas.
    #Bien, veamos como chingaos le ponemos "padding" a las convoluciones.
    medioancho=ceil(Sigma*3)
    colchon=ones(medioancho)
    result=zeros(Datos)
    datoscolchon=vcat(colchon*Datos[1], Datos, colchon*Datos[end])
    kernel=map(x->UnNormGauss(x,Sigma), [-medioancho:medioancho])
    kernel=kernel/(sum(kernel))
    #La convolucion asi normalizada preserva el valor RELATIVO entre los puntos de la funcion.
    #pero queremos ponerlo mas parecido a los voltajes que medimos, para preservar el rango de valores
    #experimentales y su criterio de potenciales de accion / ruido
    for t=medioancho+1:length(Datos)-medioancho
        result[t-medioancho]=sum(datoscolchon[t-medioancho:t+medioancho].*kernel)
    end
    a=maximum(abs(Datos))
    b=maximum(abs(result))
    result=result*a/b
    return result
end
    

GaussSuavizar (generic function with 1 method)

In [12]:
#Datos de interes general
VoltajeBase=minimum(CuernoDeAmon);
VoltajeMaximo=maximum(CuernoDeAmon);
(VoltajeBase, VoltajeMaximo)

(-149.0478515625,143.00537109375)

In [13]:
Chumbaganga=zeros(CuernoDeAmon);
for j=1:64, k=1:64
    if(MascaraOtsu[j,k]==1)
   #     Chumbaganga[j,k,:]=GaussSuavizar(reshape(CuernoDeAmon[j,k,:],tmax),7)
        Chumbaganga[j,k,:]=map(aplastar, reshape(CuernoDeAmon[j,k,:], tmax))
        Chumbaganga[j,k,:]=GaussSuavizar(reshape(Chumbaganga[j,k,:],tmax),7)
    end    
end

In [17]:
writedlm("MascaraOtsu01.dat", MascaraOtsu)

In [14]:
Chumbaganga=map(x->isnan(x)? 0:x, Chumbaganga);

In [15]:
# a escribir, pa variar
for t=1:tmax
#    aux=map(x-> isnan(x)?0:x, Chumbaganga[:,:,t])
    zacatlan="GaussianFilteredonTime-$t.dat"
    writedlm(zacatlan, Chumbaganga[:,:,t])
end


In [16]:
MascaraOtsu[50,50]

1

In [19]:
Vmaxmas=maximum(Chumbaganga);
Vminmas=minimum(Chumbaganga);
(Vmaxmas,Vminmas)

(143.00537109375,-149.0478515625)

In [17]:
GaussianKernel=readdlm("GaussianMatrixFilter01.dat");
function GaussianSmooth(Datos)
    tamanodatos=size(Datos)
    result=zeros(Datos)
    temp=copy(Datos)
    #Primero, hacemos el padding de los datos para que no se suavice demasiado
    for j=1:3
        temp=vcat(temp[1,:], temp, temp[end,:])
    end
    for j=1:3
        temp=hcat(temp[:,1], temp, temp[:,1])
    end
    
    for j=4:tamanodatos[1]+3, k=4:tamanodatos[2]+3
        #los indices van primero, "renglones", luego "columnas", etc
        aux=temp[j-3:j+3,k-3:k+3]
        result[j-3,k-3]=sum(GaussianKernel.*aux)
    end
    #piensa como normalizar
    #result=result*maximum(abs(Datos))/maximum(abs(result))
    return result
end

GaussianSmooth (generic function with 1 method)

In [20]:
@time CuackCuack=GaussianSmooth(Chumbaganga[:,:,59]);

elapsed time: 0.024252052 seconds (6912488 bytes allocated)


In [21]:
Sumbaganga=zeros(Chumbaganga)
for t=1:tmax
    Sumbaganga[:,:,t]=GaussianSmooth(Chumbaganga[:,:,t])
end


In [22]:
Sumbaganga=Sumbaganga*(Vmaxmas/maximum(Sumbaganga));

In [58]:
# y otra vez escribiendo.
for t=1:tmax
    zacatlan="GaussSmoothSpaceandTime01-$t.dat"
    writedlm(zacatlan, Sumbaganga[:,:,t])
end

In [23]:
#Un par de ejemplos para ver como se ven esas cosas:
writedlm("SuavizadoSP-21-50.dat", reshape(Sumbaganga[21,50,:],tmax))
writedlm("SuavizadoSP-50-50.dat", reshape(Sumbaganga[50,50,:],tmax))

Varias medidas estadisticas que vamos a usar, desviacion estandar total, por canal, etc.

In [24]:
extrema(Sumbaganga)

(-201.35533038269236,143.00537109375)

In [20]:
ActividadMascara=1:tmax;
for j=1:64, k=1:64
    if(MascaraOtsu[j,k]==1)
    ActividadMascara=hcat(ActividadMascara, reshape(Sumbaganga[j,k,:],tmax))
    end
end

In [27]:
map!(x->abs(x)<20?0:x, Sumbaganga);
SumbagangaPositiva=map(x->(x>0)?x:0, Sumbaganga);
SumbagangaNegativa=map(x->(x<0)?-x:0,Sumbaganga) ;

In [29]:
Salida=open("DatosCMNegativo.dat", "w");
xanterior=32.0
yanterior=32.0
for t=1:tmax
    masa=sum(SumbagangaNegativa[:,:,t]);
    xmasa=0.0
    ymasa=0.0
    for j=1:64
        xmasa+=j*sum(SumbagangaNegativa[j,:,t])
        ymasa+=j*sum(SumbagangaNegativa[:,j,t])
    end
    xmasa=xmasa/masa
    ymasa=ymasa/masa
    if(isnan(xmasa)|| isnan(ymasa))
        write(Salida,join((xanterior,yanterior,masa,t), "\t"), "\n" )
    elseif(masa<80.0)
        write(Salida,join((xanterior,yanterior,0.00,t), "\t"), "\n" )
    else
        xanterior=xmasa
        yanterior=ymasa
        write(Salida,join((xmasa,ymasa,masa,t), "\t"), "\n" )
    end
end
close(Salida)

In [21]:
map!(x->isnan(x)? 0:x, ActividadMascara);
writedlm("ActividadSuavizadaPorCanal01.dat", ActividadMascara)

In [32]:
LaplacianKernel=zeros(3,3);
#checa que Julia va ordenando los datos renglon por renglon, no columna por columna
LaplacianKernel=[[0.5 1 0.5], [1 -6 1], [0.5 1 0.5]];

function DiscreteLaplacian(Datos)
    result=zeros(Datos)
    temp=copy(Datos)
    #Primero, hacemos el padding de los datos para que no se suavice demasiado
    temp=vcat(temp[1,:], temp, temp[end,:])
    temp=hcat(temp[:,1], temp, temp[:,1])
    largo,ancho=size(Datos)
    aux=Array(Float64,(3,3))
    result=zeros(Datos)
    for j=2:largo, k=2:ancho
        #los indices van primero, "renglones", luego "columnas", etc
        aux=temp[j-1:j+1,k-1:k+1]
        result[j,k]=sum(LaplacianKernel.*aux)
    end
    return result
end

DiscreteLaplacian (generic function with 1 method)

In [34]:
TestLaplacian=zeros(LasMedicionesCrudas);
for t=1:tmax
    TestLaplacian[:,:,t]=DiscreteLaplacian(Sumbaganga[:,:,t])
    zacatlan="Laplacian-$t.dat"
    writedlm(zacatlan, TestLaplacian[:,:,t])
end


In [35]:
#checa que Julia va ordenando los datos renglon por renglon, no columna por columna


function MedianSmooth(Datos)
    MedianKernel=[[1 1 1], [1 1 1], [1 1 1]];
    result=zeros(Datos)
    temp=copy(Datos)
    #Primero, hacemos el padding de los datos para que no se suavice demasiado
    temp=vcat(temp[1,:], temp, temp[end,:])
    temp=hcat(temp[:,1], temp, temp[:,1])
    largo,ancho=size(Datos)
    aux=Array(Float64,(3,3))
    result=zeros(Datos)
    for j=2:largo, k=2:ancho
        #los indices van primero, "renglones", luego "columnas", etc
        aux=temp[j-1:j+1,k-1:k+1]
        result[j,k]=sum(MedianKernel.*aux)
    end
    result=result/9.0
    return result
end

MedianSmooth (generic function with 1 method)

In [36]:
SuavizadoMediano=zeros(Chumbaganga);
for t=1:tmax
    SuavizadoMediano[:,:,t]=MedianSmooth(Chumbaganga[:,:,t])
end

In [39]:
TestLaplacian=zeros(SuavizadoMediano);
for t=1:tmax
    TestLaplacian[:,:,t]=DiscreteLaplacian(SuavizadoMediano[:,:,t])
    zacatlan="SuavizadoTrevinoLaplacian-$t.dat"
    writedlm(zacatlan, TestLaplacian[:,:,t])
end

Voy  a considerar ahora solo los datos que estan en la mascara. Checate como Julia hace la operacion elemento por elemento!

In [38]:
zacatito=zeros(64,64)
auxiliar=zeros(64,64)
for t=1:tmax
    zacatlan="FlattSmooth-$t.dat"
    writedlm(zacatlan,SuavizadoMediano[:,:,t])
end

In [12]:
ProbandoGauss=GaussianSmooth(LasMedicionesCrudas[:,:,745]);
writedlm("DM745Gaussian01.dat", ProbandoGauss)

In [12]:
VarianciasTemporales=zeros(64,64);
for j=1:64
    for k=1:64
         VarianciasTemporales[j,k]=std(DatosInteresantes[j,k,:])
    end
end

In [14]:
writedlm("VarianciasTemporals01.dat", VarianciasTemporales);

In [15]:
MediasTemporales=zeros(64,64);
for j=1:64
    for k=1:64
         MediasTemporales[j,k]=mean(DatosInteresantes[j,k,:])
    end
end

In [16]:
writedlm("MediasTemporals01.dat", MediasTemporales);

In [17]:
sigmaTotalAbsoluta

16.515387990287376

In [39]:
promediototal=mean(DatosFiltrados)

-0.09987307054327693

In [19]:
VarianciasVentaneadas=zeros(DatosFiltrados)
for j=1:64
    for k=1:64
        ChorizoExemplo=reshape(DatosFiltrados[j,k,:],1401)
        VarianciasVentaneadas[j,k,:]=desviaporventanas(ChorizoExemplo,350)
    end
end

In [20]:
MediasVentaneadas=zeros(DatosFiltrados)
for j=1:64
    for k=1:64
        if (Mascara[j,k]==1)
            ChorizoExemplo=reshape(DatosFiltrados[j,k,:],1401)
            MediasVentaneadas[j,k,:]=mediaporventana(ChorizoExemplo,350)
        end
    end
end

In [21]:
zacatito=zeros(Float64,(64,64))
for t=1:1051
    zacatlan="MediasFiltradaLocal350-$t.dat"
    zacatito=VarianciasVentaneadas[:,:,t]
    writedlm(zacatlan,zacatito )
end

In [48]:
ActividadNegativa=zeros(DatosFiltrados);
ActividadPositiva=zeros(DatosFiltrados);
SpikeCountPositivo=zeros(64,64);
SpikeCountNegativo=zeros(64,64);
SpikeCountTotal=zeros(64,64;)
for j=1:64
    for k=1:64
        for t=1:1051
            if(abs(DatosFiltrados[j,k,t]-promediototal)>2*sigmaTotalAbsoluta)
                SpikeCountTotal[j,k]+=1;    
                if(MediasVentaneadas[j,k,t]<0)      
                    ActividadNegativa[j,k,t]=DatosFiltrados[j,k,t]
                    SpikeCountNegativo[j,k]+=1
                else
                    ActividadPositiva[j,k,t]=DatosFiltrados[j,k,t]
                    SpikeCountPositivo[j,k]+=1
                end
            end
            
        end
    end
end

In [49]:
writedlm("SpikeCountNegativo.dat", SpikeCountNegativo)
writedlm("SpikeCountPositivo.dat", SpikeCountPositivo)
writedlm("SpikeCountTotal.dat", SpikeCountTotal)

In [53]:
SpikeCountTotal[30,60],
Mascara[30,60]


(118.0,1)

In [67]:
testingsmooth=reshape(DatosFiltrados[30,60,:],1401);
writedlm("DatosExemplo3060.dat", testingsmooth);
suavizados=mediaporventana(testingsmooth,50);
writedlm("DatosSmoothed01.dat", suavizados);
suavizaditos=mediaporventana(testingsmooth,24);
suavizadisimos=mediaporventana(testingsmooth,350);
writedlm("DatospocoSmoothed01.dat", suavizaditos);
writedlm("DatosmuySmoothed01.dat", suavizadisimos);

In [66]:
desviacionexemplo=desviaporventanas(testingsmooth,50);
writedlm("DesviacionesExemplo01.dat", desviacionexemplo);

In [68]:
pocopocosuaves=mediaporventana(testingsmooth,10);
writedlm("MuypocoSuaves01.dat", pocopocosuaves);

In [41]:
zacatito=zeros(Float64,(64,64))
for t=1:1051
    zacatlan="PseudoSpikeCountNeg-$t.dat"
    zacatito=ActividadNegativa[:,:,t]
    writedlm(zacatlan,zacatito )
    zacatlan="PseudoSpikeCountPos-$t.dat"
    zacatito=ActividadPositiva[:,:,t]
    writedlm(zacatlan,zacatito )
end


In [24]:
Salida=open("DatosCMNegativo01.dat", "w");

In [25]:
for t=1:1051
    masa=sum(ActividadNegativa[:,:,t]);
    xmasa=0.0
    ymasa=0.0
    for j=1:64
        xmasa+=j*sum(ActividadNegativa[j,:,t])
        ymasa+=j*sum(ActividadNegativa[:,j,t])
    end
    xmasa=xmasa/masa
    ymasa=ymasa/masa
    write(Salida,join((xmasa,ymasa,masa), "\t"), "\n" )
end

In [26]:
close(Salida);
Salida=open("DatosCMPositivo01.dat", "w")

IOStream(<file DatosCMPositivo01.dat>)

In [27]:
for t=1:1051
    masa=sum(ActividadPositiva[:,:,t]);
    xmasa=0.0
    ymasa=0.0
    for j=1:64
        xmasa+=j*sum(ActividadPositiva[j,:,t])
        ymasa+=j*sum(ActividadPositiva[:,j,t])
    end
    xmasa=xmasa/masa
    ymasa=ymasa/masa
    write(Salida,join((xmasa,ymasa,masa), "\t"), "\n" )
end

In [28]:
close(Salida)

In [30]:
paescribirunfolded=sort(paescribir);
paescribirunfolded=diff(paescribirunfolded); #checa el metodo diff

LoadError: paescribir not defined
while loading In[30], in expression starting on line 1

In [31]:
writedlm("DiferenciasVariancias01.dat", paescribirunfolded)

LoadError: paescribirunfolded not defined
while loading In[31], in expression starting on line 1

In [32]:
otrasbins=range(0,0.005,200)
histodiff=hist(paescribirunfolded,otrasbins)
cruac=DataFrame();
cruac[1]=collect(otrasbins[1:end-1]);
cruac[2]=histodiff[2];
writetable("HistoDiffVariancias.dat", cruac, separator='\t')

LoadError: paescribirunfolded not defined
while loading In[32], in expression starting on line 2

In [33]:
Distributions.fit(Distributions.Gamma, paescribirunfolded)

LoadError: Distributions not defined
while loading In[33], in expression starting on line 1

In [34]:
sum(cruac[2])

LoadError: cruac not defined
while loading In[34], in expression starting on line 1

Okey, vamos a seguir Francos Way a partir de ahora. Veamos que endiabladas cosas esas funcionan mas o menos menos chungo

In [35]:
Mascara=zeros(Variancias)
Mascara=map(x->((x<26)&&(x>17))?1:0, Variancias);
writedlm("mascara.dat", Mascara)

LoadError: Variancias not defined
while loading In[35], in expression starting on line 1

In [36]:
?std

INFO: Loading help data...


Base.std(v[, region])

   Compute the sample standard deviation of a vector or array "v",
   optionally along dimensions in "region". The algorithm returns an
   estimator of the generative distribution's standard deviation under
   the assumption that each entry of "v" is an IID drawn from that
   generative distribution. This computation is equivalent to
   calculating "sqrt(sum((v - mean(v)).^2) / (length(v) - 1))".
   Note: Julia does not ignore "NaN" values in the computation. For
   applications requiring the handling of missing data, the
   "DataArray" package is recommended.


In [37]:
std(LasMedicionesCrudas)

80.68605707215762